In [ ]:
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, TimeoutError

def split_dataframe(df, chunk_size):
    return [df.iloc[i:i+chunk_size] for i in range(0, len(df), chunk_size)]

def process_chunk(chunk, prompt):
    try:
        chunk['ai_output'] = chunk['description'].apply(transform, prompt=prompt)
        return chunk
    except Exception as e:
        print(f"Error processing chunk: {e}")
        return None

# Split DataFrame
chunks = split_dataframe(df_des, 20)
processed_chunks = []

# Process Chunks with Timeout
with ThreadPoolExecutor() as executor:
    for i, chunk in enumerate(chunks):
        try:
            print(f"Processing chunk {i + 1}/{len(chunks)}...")
            future = executor.submit(process_chunk, chunk, prompt)
            processed_chunks.append(future.result(timeout=60))  # Timeout for each chunk
        except TimeoutError:
            print(f"Chunk {i + 1} timed out.")
        except Exception as e:
            print(f"Error in chunk {i + 1}: {e}")

# Combine Processed Chunks
processed_chunks = [chunk for chunk in processed_chunks if chunk is not None]
result_df = pd.concat(processed_chunks, ignore_index=True)

# Save Final Result
result_df.to_csv("final_output.csv", index=False)